# AI Legal Triage - RoBERTa Training

This notebook fine-tunes a RoBERTa model on the CUAD dataset for legal clause classification.

It will:
1. Set up the environment
2. Clone the GitHub repository
3. Install dependencies
4. Download and preprocess the CUAD dataset
5. Train the RoBERTa model
6. Save and package the model for download

## Check GPU Availability

In [ ]:
!nvidia-smi

## Clone the GitHub Repository

In [ ]:
!git clone https://github.com/adarench/AI-Legal-Triage.git
%cd AI-Legal-Triage

## Install Dependencies

In [ ]:
!pip install -r requirements.txt

## Create Necessary Directories

In [ ]:
!mkdir -p data/cuad_processed
!mkdir -p bert_model/fine_tuned_roberta

## Run Training with Optimal Parameters for Kaggle P100 GPU

In [ ]:
!python -m bert_model.train_model \
    --epochs 3 \
    --batch_size 16 \
    --learning_rate 3e-5 \
    --max_length 512

## Compress the Model for Download

In [ ]:
!zip -r fine_tuned_roberta.zip bert_model/fine_tuned_roberta/

## Done!

You can now download the `fine_tuned_roberta.zip` file, which contains the trained model.

Unzip it into your local project's `bert_model/` directory to use it for inference.